In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 2.7/2.7 GB 181.3 kB/s eta 0:00:00
     ---------------------------------------- 4.9/4.9 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 4.0/4.0 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 154.0/154.0 kB 3.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
!git clone https://github.com/Waer1/quizes-scrapper/

In [59]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import re
import sys
import os
import pickle
# sys.path.insert(0, os.path.abspath('../..'))
import textwrap
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torch.nn.utils.rnn import pad_sequence
import ast
tokenizer = get_tokenizer('basic_english')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elwae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elwae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elwae\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
# variables cell
# dataset_path = './quizes-scrapper/dataset'
dataset_path = '../dataset.json'


In [40]:
# Read the JSON file
df = pd.read_json(dataset_path)

# Drop rows with missing values
df = df.dropna(subset=['context', 'question', 'options'])

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function to preprocess the text
def preprocess_text(text):
    # Check if the text is a list
    if isinstance(text, list):
        # Preprocess each option in the list
        return [preprocess_option(option) for option in text]
    else:
        # Preprocess the text
        return preprocess_option(text)

def preprocess_option(option):
    # Tokenize the text
    tokens = word_tokenize(option)
    # Remove stopwords and lemmatize the words
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords.words('english')]
    # Convert the tokens into lowercase
    tokens = [token.lower() for token in tokens]
    # Remove non-alphabetic characters from tokens
    tokens = [token for token in tokens if token.isalpha()]
    # Join the tokens together
    return ' '.join(tokens)

# Preprocess the text
df['context'] = df['context'].apply(preprocess_text)
df['question'] = df['question'].apply(preprocess_text)
df['options'] = df['options'].apply(preprocess_text)
df['answer'] = df['answer'].apply(preprocess_text)

# Split the data into a training set and a test set
train_df, test_df = train_test_split(df, test_size=0.2)

In [65]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):

        #src = [src len, batch size]

        embedded = self.dropout(self.embedding(src))

        #embedded = [src len, batch size, emb dim]

        outputs, (hidden, cell) = self.rnn(embedded)

        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #outputs are always from the top hidden layer

        return hidden, cell
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):

        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #input = input.unsqueeze(0)

        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))

        #embedded = [1, batch size, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))

        #prediction = [batch size, output dim]

        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, input_dim, output_dim, enc_emb_dim, dec_emb_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.encoder = Encoder(input_dim, enc_emb_dim, hidden_dim, n_layers, dropout)
        self.decoder = Decoder(output_dim, dec_emb_dim, hidden_dim, n_layers, dropout)

    def forward(self, src, trg):
        # src = [src len, batch size]
        # trg = [trg len, batch size]
        enc_output, hidden = self.encoder(src)
        output = self.decoder(trg, hidden, enc_output)
        return output


# Create an instance of the Seq2Seq model
model = Seq2Seq(INPUT_DIM, OUTPUT_DIM, ENC_EMB_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)

In [66]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Number of epochs
n_epochs = 10

# Training loop
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for i, (src, trg) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = model(src, trg)
        loss = criterion(output.view(-1, output.shape[-1]), trg.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch: {epoch}, Loss: {total_loss / len(train_dataloader)}")

RuntimeError: Expected hidden[0] size (2, 237, 512), got [2, 8, 512]

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader, Dataset
from transformers.data.datasets.language_modeling import LineByLineTextDataset
import pandas as pd
import os


# variables cell

dataset_path = './quizes-scrapper/dataset.json'


# Fine-tuning parameters
num_epochs = 3
learning_rate = 1e-5


# Load pre-trained BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Read the JSON file
df = pd.read_json(dataset_path)

contexts = df['context'].tolist()
questions = df['question'].tolist()
options = df['options'].tolist()
answers = df['answer'].tolist()
explanations = df['explanation'].tolist()
code_blocks = df['codeblock'].tolist()

class CustomDataset(LineByLineTextDataset):
    def __init__(self, tokenizer, df, block_size=512):
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        context = self.df.iloc[idx]['context']
        question = self.df.iloc[idx]['question']
        answer = self.df.iloc[idx]['answer']

        # Combine context, question, and answer into a single string
        text = f"context: {context} question: {question} answer: {answer}"

        # Tokenize the text
        input_ids = self.tokenizer.encode(text, return_tensors='pt', max_length=self.block_size, truncation=True)

        return {"input_ids": input_ids.squeeze()}

# Create dataset and DataLoader
custom_dataset = CustomDataset(tokenizer, df)

# Create DataLoader with a collator that pads sequences to the maximum length in the batch
data_collator = DataCollatorForLanguageModeling(tokenizer)
dataloader = DataLoader(custom_dataset, batch_size=4, shuffle=True, collate_fn=data_collator)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)
!export TORCH_USE_CUDA_DSA=1


# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)


# Fine-tuning loop
for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        labels = input_ids.clone()  # Create labels before shifting

        # Shift the input_ids to the right
        input_ids = torch.cat([torch.full_like(input_ids[:, :1], tokenizer.pad_token_id), input_ids[:, :-1]], dim=-1)

        # Replace eos_token_id in labels with -100
        labels[labels==tokenizer.eos_token_id] = -100

        # Check the maximum token ID in the labels tensor
        print(f"Max token ID in labels: {labels.max().item()}")

        # Check the size of the target vocabulary
        print(f"Size of target vocabulary: {model.config.vocab_size}")

        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

    scheduler.step()

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader, Dataset
from transformers.data.datasets.language_modeling import LineByLineTextDataset
import pandas as pd
import os


# variables cell

dataset_path = './quizes-scrapper/dataset.json'


# Fine-tuning parameters
num_epochs = 3
learning_rate = 1e-5


# Load pre-trained BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Read the JSON file
df = pd.read_json(dataset_path)

contexts = df['context'].tolist()
questions = df['question'].tolist()
options = df['options'].tolist()
answers = df['answer'].tolist()
explanations = df['explanation'].tolist()
code_blocks = df['codeblock'].tolist()

class CustomDataset(LineByLineTextDataset):
    def __init__(self, tokenizer, df, block_size=512):
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        context = self.df.iloc[idx]['context']
        question = self.df.iloc[idx]['question']
        answer = self.df.iloc[idx]['answer']

        # Combine context, question, and answer into a single string
        text = f"context: {context} question: {question} answer: {answer}"

        # Tokenize the text
        input_ids = self.tokenizer.encode(text, return_tensors='pt', max_length=self.block_size, truncation=True)

        return {"input_ids": input_ids.squeeze()}

# Create dataset and DataLoader
custom_dataset = CustomDataset(tokenizer, df)

# Create DataLoader with a collator that pads sequences to the maximum length in the batch
data_collator = DataCollatorForLanguageModeling(tokenizer)
dataloader = DataLoader(custom_dataset, batch_size=4, shuffle=True, collate_fn=data_collator)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)
!export TORCH_USE_CUDA_DSA=1


# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# Fine-tuning loop
for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)

        # Check the maximum token ID in the labels tensor
        print(f"Max token ID in labels: {labels.max().item()}")

        # Check the size of the target vocabulary
        print(f"Size of target vocabulary: {model.config.vocab_size}")

        labels = input_ids[:, 1:].contiguous()
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

    scheduler.step()


In [ ]:
        context = self.df.iloc[idx]['context']
        question = self.df.iloc[idx]['question']
        answer = self.df.iloc[idx]['answer']
        explanation = self.df.iloc[idx]['explanation']
        code_block = self.df.iloc[idx]['codeblock']


        # Combine context, question, and answer into a single string
text = f"context: {context} question: {question} answer: {answer}"
if explanation is not None:
    text += f" explanation: {explanation}"
if code_block is not None:
    text += f" code_block: {code_block}"